# Линейный SVM "своими руками"

##Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.cross_validation import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(n_samples=10000, n_features=20, n_classes=2, n_informative=20, n_redundant=0,
                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

print len(X), len(y)
print len(X_train)

10000 10000
8000


/Users/daniilkorbut/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Пишем свой класс для SVM

In [88]:
import numpy as np
from random import randint
import random

np.random.seed(42)
random.seed(42)

class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        1.0 / self.C * (self.w)
        '''ToDo: fix this function'''
        return 0.0

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        return -1.0 if 1 - answer * self.f(x) > 0 else 0.0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            '''ToDo: add w update with regularization'''
            self.w -= step * (y * x * self.der_loss(x,y) + self.der_reg())
            # w_0 update
            '''ToDo: add w_0 update'''
            self.w0 -= step * (y * self.der_loss(x,y))

In [89]:
x = np.array([1,2,3])
y = np.array([5,6,7])
x * y

array([ 5, 12, 21])

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [90]:
model = MySVM()
model.fit(X_train, y_train)
print model.w, model.w0

[ 0.95752876 -0.74588476  0.04693989  1.34613313 -0.10414925  1.64820727
 -0.15097709 -0.30911416  0.15622769 -2.78075828 -0.10142291 -0.32634712
  0.91655727 -0.2705666  -1.64558011 -1.10065976  1.62517089  1.13768769
  1.38027198  1.57419087] -0.943505480738


In [91]:
predictions = model.predict(X_test)

In [92]:
print predictions

[1 0 0 ..., 1 1 1]


In [93]:
print y_test, len(y_test), sum(y_test)

[1 0 1 ..., 1 0 1] 2000 991


In [94]:
print len(predictions), sum(predictions)

2000 1103


In [95]:
print sum(predictions == y_test) / float(len(y_test))

0.678
